    %load_ext pidgin.shell
    
The `pidgin` shell augments the `IPython` shell in the notebook to permit Markdown forward [computational essays](http://blog.stephenwolfram.com/2017/11/what-is-a-computational-essay/).   It provides:

* Markdown input in code cells.
* Markdown block code statements and expressions are evaluated as a normal code cell.
* Inline code expressions are evaluated asynchronously.
* The Markdown code accepts `jinja2` templates too


    IPython.core.interactiveshell.InteractiveShell._run_cell??

In [1]:
    import IPython, traitlets, jinja2, types, contextlib, textwrap, ast, argparse, traceback, doctest, sys, traceback
    try:
        from .tangle import markdown_to_python, markdown_to_expressions
    except:
        from tangle import markdown_to_python, markdown_to_expressions

2. `run_cell_doctests` _will discover any doctests in the cell and evaluate them_.
3. `run_user_expressions` _executes inline code expressions to assure that it is valid code_.
    
    __assert__ is the only statement permitted in inline code.
4. `run_cell_display` _display the input_.

In [2]:
    import functools

In [3]:
    def show_error(ip, object):
        @functools.wraps(object)
        def post_run_cell(result):
            if result.error_in_exec or result.error_before_exec: return result
            result.error_in_exec = object(result)
            if result.error_in_exec:
                ip.showtraceback(exc_to_tuple(result.error_in_exec))
            return result
        return post_run_cell

`exc_to_tuple` converts an `Exception` to a `tuple` to display the traceback__.

In [4]:
    def exc_to_tuple(Exception): return type(Exception), Exception, Exception.__traceback__

## `run_cell_doctests`

In [5]:
    def run_cell_doctests(ip, result) -> Exception:
        with IPython.utils.capture.capture_output() as output:
            doctest.run_docstring_examples(result.info.raw_cell, ip.user_ns, name=ip.user_ns['__name__'])

        if output.stdout:
            return DocTestException(output.stdout)

In [6]:
    class DocTestException(BaseException):
        def _render_traceback_(self): return args[0].splitlines()

## `run_user_expressions` 

In [7]:
    def run_user_expressions(ip, result):
        try: user_expressions(ip, {str: str for str in markdown_to_expressions(result.info.raw_cell)})
        except BaseException as Exception: return Exception

In [8]:
    def user_expressions(ip, expressions):
        """User expressions evaluate expressions, assert statements are the only exception. `assert` may
        augment a literate narrative so we choose to include these statements in the expressions.
        """
        for key, expression in expressions.items():
            expression = ip.input_transformer_manager.transform_cell(textwrap.indent(expression, ' '*4))
            try:
                exec(expression, ip.user_global_ns, ip.user_ns)
            except BaseException as Exception:
                raise InlineCodeException(expression)

In [9]:
    class InlineCodeException(BaseException): ...

## `run_cell_display`

In [41]:
    def run_cell_display(ip, result) -> bool:
        raw, source = result.info.raw_cell, markdown_to_python(result.info.raw_cell)
        try:
            is_display = bool(raw.partition('\n')[0].strip() and not raw.startswith(';')) 
            if raw.strip() == ast.literal_eval(source.rstrip().rstrip(';')).strip():
                is_display = False
        except: ...
        if is_display and not only_contains_block_code(raw, source):
            try: IPython.display.display(raw)
            except BaseException as Exception: return Exception 

`only_contains_block_code` determines if the input source and python only include block code.

In [11]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
        return list(
            map(str.rstrip, textwrap.dedent(raw).splitlines())
        ) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

# Extensions

In [34]:
    def load_ipython_extension(ip): 
        _show_error = types.MethodType(show_error, ip)
        for callable in (run_cell_doctests, run_user_expressions, run_cell_display):
            ip.events.register('post_run_cell', _show_error(types.MethodType(callable, ip)))

    def unload_ipython_extension(ip): 
        ip.events.callbacks['post_run_cell'] = [
            object for object in ip.events.callbacks['post_run_cell']
            if object.__name__ not in {
                'run_cell_doctests', 'run_user_expressions', 'run_cell_display'}]